## Newton's Forward Interpolation

In [1]:
def comb(n, i):
    num = 1
    den = 1

    for j in range(0, i):
        num *= (n-j)
        den *= j+1
    
    return num/den

def newton_forward_interpolation(x, y, x_pred):
    """
    Arguments
    ---------
    x: list
        x values
    y: list
        y value
    x_pred: float
        x value for which y value is being predicted
    """
    if len(x) != len(y):
        raise ValueError("x and y shape don't match")

    n = len(x)
    h = x[1] - x[0]         # Assuming equidistant x values
    u = (x_pred - x[0])/h

    # Maintaining a column of forward difference table
    # [Move ahead in the table in each iteration]
    fd_col = list(y)

    y_pred = 0
    for i in range(0, n):
        y_pred += comb(u, i) * fd_col[0]
        for j in range(0, n-i-1):
            fd_col[j] = fd_col[j+1] - fd_col[j]
    
    return y_pred

In [2]:
# Set of distint points
x = [1, 1.02, 1.04, 1.06, 1.08]
y = [0.242, 0.2371, 0.2323, 0.2275, 0.2227]

# Find interpolated value for
x_pred = 1.015

newton_forward_interpolation(x, y, x_pred)

0.23830952148437504

## Newton's Backward Interpolation

In [3]:
def newton_backward_interpolation(x, y, x_pred):
    """
    Arguments
    ---------
    x: list
        x values
    y: list
        y value
    x_pred: float
        x value for which y value is being predicted
    """
    if len(x) != len(y):
        raise ValueError("x and y shape don't match")

    n = len(x)
    h = x[1] - x[0]         # Assuming equidistant x values
    u = (x_pred - x[-1])/h

    # Maintaining a column of (reverse) backward difference table
    # [Move ahead in the table in each iteration]
    rbk_col = y[::-1]   # reverse
    print()

    y_pred = 0
    for i in range(0, n):
        y_pred += (-1)**i * comb(-u, i) * rbk_col[0]
        for j in range(0, n-i-1):
            rbk_col[j] = rbk_col[j] - rbk_col[j+1]
    
    return y_pred

In [4]:
# Set of distint points
x = [1, 1.05, 1.10, 1.15, 1.20, 1.25]
y = [0.682689, 0.706282, 0.728668, 0.749856, 0.769861, 0.788700]

# Find interpolated value for
x_pred = 1.235

newton_backward_interpolation(x, y, x_pred)

0.7831697570560001

## Newton's Divided Difference Interpolation

In [5]:
def newton_divided_difference_interpolation(x, y, x_pred):
    """
    Arguments
    ---------
    x: list
        x values
    y: list
        y value
    x_pred: float
        x value for which y value is being predicted
    """
    if len(x) != len(y):
        raise ValueError("x and y shape don't match")

    n = len(x)

    # Maintaining a column of divided difference table
    # [Move ahead in the table in each iteration]
    dd_col = list(y)

    x_coef = 1
    y_pred = 0
    for i in range(0, n):
        y_pred += x_coef * dd_col[0]
        x_coef *= (x_pred - x[i])
        for j in range(0, n-i-1):
            dd_col[j] = (dd_col[j+1] - dd_col[j])/(x[j+i+1] - x[j])
    
    return y_pred

In [6]:
# Set of distint points
x = [0.5, 1.5, 3.0, 5.0, 6.5, 8.0]
y = [1.625, 5.875, 31.0, 131.0, 282.125, 521.0]

# Find interpolated value for
x_pred = 7

newton_divided_difference_interpolation(x, y, x_pred)

351.0

## Langrange's Interpolation

In [18]:
def L(n, k):
    """
    Lagrange Coefficient Polynomial
    """
    num = 1
    den = 1
    for i in range(n):
        if i != k:
            num *= (x_pred - x[i])
            den *= (x[k] - x[i])
    return num/den

def lagrange_interpolation(x, y, x_pred):
    """
    Arguments
    ---------
    x: list
        x values
    y: list
        y value
    x_pred: float
        x value for which y value is being predicted
    """
    if len(x) != len(y):
        raise ValueError("x and y shape don't match")

    n = len(x)
    y_pred = 0
    for i in range(0, n):
        y_pred += y[i] * L(n, i)
    
    return y_pred

In [19]:
# Set of distint points
x = [300, 304, 305, 307]
y = [2.4771, 2.4829, 2.4843, 2.4871]

# Find interpolated value for
x_pred = 301

lagrange_interpolation(x, y, x_pred)

2.4785971428571423

## Hermite Interpolation

Using Newton's divided difference approximation

In [51]:
def double_each_element(l):
    n = len(l)
    l_new = []
    for i in range(n):
        l_new.append(l[i])
        l_new.append(l[i])
    return l_new

def hermite_interpolation(x, y, y_red, x_pred):
    """
    Arguments
    ---------
    x: list
        x values
    y: list
        y value
    y_der: list
        y derivative values
    x_pred: float
        x value for which y value is being predicted
    """
    if len(x) != len(y):
        raise ValueError("x and y shape don't match")

    z = double_each_element(x)
    n = len(z)

    # Maintaining a column of divided difference table
    # [Move ahead in the table in each iteration]
    dd_col = double_each_element(y)
    for j in range(0, n-1):
        if j%2 == 0:
            dd_col[j] = y_der[j//2]
        else:
            dd_col[j] = (dd_col[j+1] - dd_col[j])/(z[j+1] - z[j])

    z_coef = (x_pred - z[0])
    y_pred = y[0]
    for i in range(1, n):
        y_pred += z_coef * dd_col[0]
        z_coef *= (x_pred - z[i])
        for j in range(0, n-i-1):
            dd_col[j] = (dd_col[j+1] - dd_col[j])/(z[j+i+1] - z[j])
    
    return y_pred


In [52]:
# Set of distint points
x = [1.3, 1.6, 1.9]
y = [0.6200860, 0.4554022, 0.2818186]
y_der = [-0.5220232, -0.5698958, -0.5811571]

# Find interpolated value for
x_pred = 1.5

hermite_interpolation(x, y, y_der, x_pred)

0.5118276938271605